In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="7"
from sklearn.preprocessing import LabelBinarizer

import shfl

/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/frame

In [2]:
#"../data/COVIDGR1.0/centralized/cropped"

args = {"data_path":"../data/COVIDGR1.0-cropped", 
        "csv_path": "../partitions/partition_iid_3nodes_1.csv",
        "output_path": "../weights",
        "input_path": "",
        "model_name":"transferlearning.model", 
        "label_bin": "lb.pickle", 
        "batch_size": 8,
        "federated_rounds": 2,
        "epochs_per_FL_round": 100,
        "num_nodes": 3,
        "size_averaging": 1,
        "random_rotation": 0,
        "random_shift": 0, 
        "random_zoom": 0,
        "horizontal_flip": False,        
        "finetune": True,
        "train_network": True}

In [3]:
a = ['N', 'P']
b = ['NTN', 'NTP', 'PTP', 'PTN']

lb1 = LabelBinarizer()
lb2 = LabelBinarizer()

lb1.fit(a)
lb2.fit(b)

LabelBinarizer()

In [5]:
LABELS = ["N", "P"]
print("[INFO] training for labels: " + str(LABELS))

print("[INFO] Distributing the train set across the nodes...")
federated_data, test_data, test_label, train_files, test_files = get_federated_data_csv(args["data_path"], args["csv_path"])
print("[INFO] done")

[INFO] training for labels: ['N', 'P']
[INFO] Distributing the train set across the nodes...


/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/ipykernel_launcher.py:46: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/ipykernel_launcher.py:52: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


[INFO] done


from shfl.private import UnprotectedAccess

print(test_data.shape)
print(test_label.shape)
print(federated_data.num_nodes())
federated_data.configure_data_access(UnprotectedAccess())
a = federated_data[0].query()._data
print(test_label)


In [6]:
from CITModel import CITModel
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def model_builder():    
    return CITModel(LABELS, classifier_name = "resnet18", lambda_value = 0.00075, batch_size=args["batch_size"], epochs=args["epochs_per_FL_round"], device=device)

In [ ]:
aggregator = shfl.federated_aggregator.FedAvgAggregator()
federated_government = shfl.federated_government.FederatedGovernment(model_builder, federated_data, aggregator)
federated_government.run_rounds(args["federated_rounds"], test_data, test_label)

[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING
[INFO] FREEZING


  0%|          | 0/18 [00:00<?, ?it/s]

Accuracy round 0
[INFO] weights = [1.         0.95714286]


[0/50] Loss_D: 0.5119 Acc_D: 0.5219 Loss_G_class1: 3918.4218 Loss_G_class2: 4099.0678: 100%|██████████| 18/18 [00:24<00:00,  1.39s/it]
[1/50] Loss_D: 0.3374 Acc_D: 0.5730 Loss_G_class1: 792.9886 Loss_G_class2: 938.9191: 100%|██████████| 18/18 [00:24<00:00,  1.37s/it]
[2/50] Loss_D: 0.3069 Acc_D: 0.6533 Loss_G_class1: 571.9469 Loss_G_class2: 713.1103: 100%|██████████| 18/18 [00:24<00:00,  1.37s/it]
[3/50] Loss_D: 0.2727 Acc_D: 0.7299 Loss_G_class1: 460.0826 Loss_G_class2: 593.5011: 100%|██████████| 18/18 [00:24<00:00,  1.36s/it]
[4/50] Loss_D: 0.2341 Acc_D: 0.7847 Loss_G_class1: 384.8871 Loss_G_class2: 507.7466: 100%|██████████| 18/18 [00:24<00:00,  1.37s/it]
[5/50] Loss_D: 0.1802 Acc_D: 0.8467 Loss_G_class1: 318.8187 Loss_G_class2: 428.9515: 100%|██████████| 18/18 [00:24<00:00,  1.37s/it]
[6/50] Loss_D: 0.1913 Acc_D: 0.8029 Loss_G_class1: 261.8874 Loss_G_class2: 362.4457: 100%|██████████| 18/18 [00:24<00:00,  1.37s/it]
[7/50] Loss_D: 0.1122 Acc_D: 0.9124 Loss_G_class1: 211.6623 Loss_G_

[INFO] weights = [1.         0.78947368]


...

[0/50] Loss_D: 0.4609 Acc_D: 0.5803 Loss_G_class1: 9.0614 Loss_G_class2: 12.1506: 100%|██████████| 18/18 [00:24<00:00,  1.37s/it]
[1/50] Loss_D: 0.3180 Acc_D: 0.6277 Loss_G_class1: 8.6728 Loss_G_class2: 11.6156: 100%|██████████| 18/18 [00:24<00:00,  1.37s/it]
[2/50] Loss_D: 0.2605 Acc_D: 0.7591 Loss_G_class1: 8.2997 Loss_G_class2: 11.0590: 100%|██████████| 18/18 [00:24<00:00,  1.37s/it]
[3/50] Loss_D: 0.2668 Acc_D: 0.7847 Loss_G_class1: 7.9527 Loss_G_class2: 10.6106: 100%|██████████| 18/18 [00:24<00:00,  1.37s/it]
[4/50] Loss_D: 0.1950 Acc_D: 0.8066 Loss_G_class1: 7.6174 Loss_G_class2: 10.1734: 100%|██████████| 18/18 [00:24<00:00,  1.37s/it]
[5/50] Loss_D: 0.0993 Acc_D: 0.9124 Loss_G_class1: 7.2817 Loss_G_class2: 9.7664: 100%|██████████| 18/18 [00:24<00:00,  1.37s/it]
[6/50] Loss_D: 0.0415 Acc_D: 0.9745 Loss_G_class1: 6.9621 Loss_G_class2: 9.3789: 100%|██████████| 18/18 [00:24<00:00,  1.37s/it]
[7/50] Loss_D: 0.0233 Acc_D: 0.9854 Loss_G_class1: 6.6680 Loss_G_class2: 9.0116: 100%|██████

[INFO] weights = [1.         0.78947368]


[0/50] Loss_D: 0.4513 Acc_D: 0.5588 Loss_G_class1: 8.6641 Loss_G_class2: 11.6545: 100%|██████████| 17/17 [00:24<00:00,  1.43s/it]
[1/50] Loss_D: 0.4069 Acc_D: 0.5772 Loss_G_class1: 8.2749 Loss_G_class2: 11.1131: 100%|██████████| 17/17 [00:24<00:00,  1.44s/it]
[2/50] Loss_D: 0.2353 Acc_D: 0.7684 Loss_G_class1: 7.9124 Loss_G_class2: 10.6069: 100%|██████████| 17/17 [00:24<00:00,  1.44s/it]
[3/50] Loss_D: 0.2442 Acc_D: 0.7904 Loss_G_class1: 7.5643 Loss_G_class2: 10.1682: 100%|██████████| 17/17 [00:24<00:00,  1.44s/it]
[4/50] Loss_D: 0.1821 Acc_D: 0.8162 Loss_G_class1: 7.2401 Loss_G_class2: 9.7533: 100%|██████████| 17/17 [00:24<00:00,  1.44s/it]
[5/50] Loss_D: 0.0778 Acc_D: 0.9779 Loss_G_class1: 6.9327 Loss_G_class2: 9.3589: 100%|██████████| 17/17 [00:24<00:00,  1.44s/it]
[6/50] Loss_D: 0.0163 Acc_D: 0.9926 Loss_G_class1: 6.6419 Loss_G_class2: 8.9854: 100%|██████████| 17/17 [00:24<00:00,  1.44s/it]
[7/50] Loss_D: 0.0083 Acc_D: 1.0000 Loss_G_class1: 6.3667 Loss_G_class2: 8.6297: 100%|███████

[INFO] weights = [1.         0.94285714]


[0/50] Loss_D: 0.4150 Acc_D: 0.5882 Loss_G_class1: 9.2934 Loss_G_class2: 12.3981: 100%|██████████| 17/17 [00:24<00:00,  1.44s/it]
[1/50] Loss_D: 0.3384 Acc_D: 0.7132 Loss_G_class1: 8.8768 Loss_G_class2: 11.7947: 100%|██████████| 17/17 [00:24<00:00,  1.44s/it]
[2/50] Loss_D: 0.2709 Acc_D: 0.7794 Loss_G_class1: 8.4736 Loss_G_class2: 11.2852: 100%|██████████| 17/17 [00:24<00:00,  1.44s/it]
[3/50] Loss_D: 0.1978 Acc_D: 0.8640 Loss_G_class1: 8.0964 Loss_G_class2: 10.8032: 100%|██████████| 17/17 [00:24<00:00,  1.44s/it]
[4/50] Loss_D: 0.1007 Acc_D: 0.9338 Loss_G_class1: 7.7408 Loss_G_class2: 10.3569: 100%|██████████| 17/17 [00:24<00:00,  1.43s/it]
[5/50] Loss_D: 0.0733 Acc_D: 0.9286 Loss_G_class1: 7.5786 Loss_G_class2: 10.1693:  82%|████████▏ | 14/17 [00:20<00:04,  1.41s/it]

In [ ]:
from shfl.private import UnprotectedAccess
from torchvision import transforms
from torchvision.transforms import ToPILImage
from torch.autograd import Variable
import matplotlib.pyplot as plt
from IPython.display import display
from PIL import Image
import cv2
from sklearn.preprocessing import LabelBinarizer

import copy


def sample_loader(sample):
    loader = transforms.Compose([transforms.ToTensor()])
    s = loader(sample).float()
    s = Variable(s, requires_grad=False)
    s = s.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    return s.to(device) #assumes that you're using GPU

def show_img(G_dict, sample):
    x = sample_loader(sample)
    class_name = LABELS[0]
    y = G_dict[class_name](x)
    y = ToPILImage()(y[0].cpu().detach())
    #y.save("./prueba.png")
    display(y)
    
def transform_data(G_dict, class_names, data, labels):
    new_labels = []
    new_data = []
    for i in range(len(data)):
        sample = data[i]
        label = lb1.inverse_transform(labels[i])[0]
        x = sample_loader(sample)
        for class_name in class_names:
            y = G_dict[class_name](x)
            y = y[0].cpu().detach().numpy()
            y = np.moveaxis(y, 0, -1)
            new_data.append(y)
            new_label = str(label) + "T" + class_name
            new_labels.append(new_label)
    new_labels = lb2.transform(new_labels)
    
    return np.asarray(new_data), np.asarray(new_labels)


G_dict = federated_government.global_model._G_dict
for class_name in LABELS:
    G_dict[class_name]= G_dict[class_name].to(device)
federated_data.configure_data_access(UnprotectedAccess())

new_federated_data = copy.deepcopy(federated_data)

for i in range(federated_data.num_nodes()):
    data_node = federated_data[i]
    new_data_node = new_federated_data[i]
    data = data_node.query()._data
    labels = data_node.query()._label
    new_data, new_labels = transform_data(G_dict, LABELS, data, labels)
    new_data_node.query()._data = new_data
    new_data_node.query()._label = new_labels
    
    #print(data_node.query()._label)
    #print(new_data_node.query()._label)
    #data_node.query()._data = new_data
    #data_node.query()._label = new_labels

new_test_data, new_test_label = transform_data(G_dict, LABELS, test_data, test_label)

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input

import tensorflow as tf


"""
datagen_train = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    rotation_range = args["random_rotation"],
    width_shift_range = args["random_shift"],
    height_shift_range = args["random_shift"],
    zoom_range = args["random_zoom"],
    horizontal_flip = args["horizontal_flip"]
)
"""

train_datagen = ImageDataGenerator(preprocessing_function = preprocess_input, validation_split=0.1)



class TransferLearningModel(shfl.model.DeepLearningModel):    
    
    def train(self, data, labels):
        train_generator = train_datagen.flow(data, labels, batch_size=args["batch_size"], subset='training')

        validation_generator = train_datagen.flow(data, labels, batch_size=args["batch_size"], subset='validation')
        #self._check_data(data)
        #self._check_labels(labels)

        #early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')
        self._model.fit(
            x=train_generator,
            steps_per_epoch= int(len(data)*0.8) // args["batch_size"],
            validation_data = validation_generator,
            validation_steps = int(len(data)*0.2) // args["batch_size"],
            epochs=self._epochs
        )

def model_builder_2():
    
    resnet50 = tf.keras.applications.ResNet50(include_top = False, weights = 'imagenet', pooling = 'avg', input_tensor=Input(shape=(256, 256, 3)))
    
    if (args["finetune"]):
        resnet50.trainable = False
    else: 
        resnet50.trainable = True
    
    # Add last layers
    x = resnet50.output
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    predictions = tf.keras.layers.Dense(4, activation = 'softmax')(x)
    
    model = tf.keras.Model(inputs = resnet50.input, outputs = predictions)
    
    criterion = tf.keras.losses.CategoricalCrossentropy()
    optimizer = tf.keras.optimizers.SGD(lr = 1e-3, decay = 1e-6, momentum = 0.9, nesterov = True)
    metrics = [tf.keras.metrics.categorical_accuracy]
    
    return TransferLearningModel(model=model, criterion=criterion, optimizer=optimizer, metrics=metrics, epochs=args["epochs_per_FL_round"], batch_size = args["batch_size"])

In [ ]:
from tensorflow.keras.models import load_model

# Train the network:
epochs_per_FL_round=args["epochs_per_FL_round"]
aggregator = shfl.federated_aggregator.FedAvgAggregator()
new_federated_government = shfl.federated_government.FederatedGovernment(model_builder_2, new_federated_data, aggregator)
new_federated_government.run_rounds(args["federated_rounds"], new_test_data, new_test_label)
#print("[INFO] saving model ...")
#federated_government.global_model._model.save( os.path.join(args["output_path"], args["model_name"]) )
print("[INFO] done")

In [ ]:
from sklearn.metrics import classification_report

true_labels = []
preds = []
no_concuerda = 0
preds_4 = []
true_labels_4 = []
tabla_preds = np.empty((len(test_files), 3), dtype = '<U50')

model = new_federated_government.global_model._model

dict_labels = { 'PTP' : np.argmax(lb2.transform(['PTP'])[0]) , 'PTN' : np.argmax(lb2.transform(['PTN'])[0]) , 
                'NTP' : np.argmax(lb2.transform(['NTP'])[0]) , 'NTN' : np.argmax(lb2.transform(['NTN'])[0])
              } 

for i in range(len(test_files)):
    image_path = test_files[0]
    name = image_path.split(os.path.sep)[-1].split('.')[0]
    label = image_path.split(os.path.sep)[-2]
    
    true_labels.append(label)
    true_labels_4.append(dict_labels[label + "TP"])
    true_labels_4.append(dict_labels[label + "TN"])
    tabla_preds[i,0] = name
    
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (256, 256))
    
    x = sample_loader(image)
    tp = G_dict['P'](x)
    tp = tp[0].cpu().detach().numpy()
    tp = np.moveaxis(tp, 0, -1)
    
    tn = G_dict['N'](x)
    tn = tn[0].cpu().detach().numpy()
    tn = np.moveaxis(tn, 0, -1)

    tp = np.expand_dims(tp, axis = 0)
    tn = np.expand_dims(tn, axis = 0)
    tp = preprocess_input(tp)
    tn = preprocess_input(tn)

    prob_tp = model.predict(tp)
    prob_tn = model.predict(tn)
    
    pred_tp = np.argmax(prob_tp)
    pred_tn = np.argmax(prob_tn)
    preds_4.append(pred_tp)
    preds_4.append(pred_tn)
    
    
    # print('prediccion tp: ' + str(pred_tp))
    # print('prediccion tn: ' + str(pred_tn))

    if pred_tp == dict_labels['NTP'] and pred_tn == dict_labels['NTN']:
        pred = 'N'
    elif pred_tp == dict_labels['PTP'] and pred_tn == dict_labels['PTN']:
        pred = 'P'
    else:
        no_concuerda = no_concuerda + 1
        # prob_p = prob_tp[0][dict['PTP']] + prob_tp[0][dict['PTN']] + prob_tn[0][dict['PTP']] + prob_tn[0][dict['PTN']]
        # prob_n = prob_tp[0][dict['NTP']] + prob_tp[0][dict['NTN']] + prob_tn[0][dict['NTP']] + prob_tn[0][dict['NTN']]
        prob_p = max(prob_tp[0][dict_labels['PTP']], prob_tp[0][dict_labels['PTN']], prob_tn[0][dict_labels['PTP']], prob_tn[0][dict_labels['PTN']])
        prob_n = max(prob_tp[0][dict_labels['NTP']], prob_tp[0][dict_labels['NTN']], prob_tn[0][dict_labels['NTP']], prob_tn[0][dict_labels['NTN']])
        if prob_p >= prob_n:
            pred = 'P'
        else:
            pred = 'N'

    preds.append(pred)

true_labels = np.array(true_labels)
preds = np.array(preds)
true_labels_4 = np.array(true_labels_4)
preds_4 = np.array(preds_4)

print(preds)
print(true_labels)

tabla_preds[:,1] = true_labels
tabla_preds[:,2] = preds
#np.savetxt(save_preds_file, tabla_preds, fmt = '%1s', delimiter = ',')

# Calculate accuracy
acc_4 = sum(true_labels_4 == preds_4)/len(true_labels_4)
print('Accuracy 4 clases: ' + str(acc_4))
print('Numero de veces no concuerda: ' + str(no_concuerda))
acc = sum(true_labels == preds)/len(true_labels)
results = classification_report(true_labels, preds, digits = 5, output_dict = True)

if results['N']['recall'] >= 0.73 and results['P']['recall'] >= 0.73:
    model.save(save_model_file)

print(results)
print(acc)